# Matrix Factorization
* Prediction is $\tilde R = UA^T$
* Loss fuction is $L = \lVert (R - \tilde R)^\Omega \odot W \rVert _2^2 + \lambda_u \lVert U \rVert _2^2 + \lambda_a \lVert A \rVert _2^2$
* $\Omega$ is the set of oberved pairs $(i, j)$
* $M^\Omega$ is the projection of $M$ onto $\Omega$ for any matrix $M$, that is $M_{ij}^\Omega$ is defined to be $M_{ij}$ when $(i, j) \in \Omega$ and $0$ otherwise
* $U$ is an $m x k$ matrix, $A$ is an $n x k$ matrix and $R$ is the $m x n$ ratings matrix
* $W = w_{ij}$ is the weight for the prediction $r_{ij}$ and is modeled as a power-law in the number of items seen by $i$ and users than have seen $j$: $w_{ij} = |j' : (i, j') \in \Omega| ^ {\lambda_{wu}} |i' : (i', j) \in \Omega| ^ {\lambda_{wa}}$

In [ ]:
const name = "MatrixFactorization"
const residual_alphas = ["UserItemBiases"]
const implicit = false;

In [ ]:
using Random
using SparseArrays
import NBInclude: @nbinclude
@nbinclude("Alpha.ipynb");

In [ ]:
# TODO maybe we can delete this with a newer julia version
# Some sparse matrix operations require indices to be Int64
@with_kw struct RatingsDataset64
    user::Vector{Int64}
    item::Vector{Int64}
    rating::Vector{Float32}
end

function RatingsDataset64(x::RatingsDataset)
    RatingsDataset64(
        convert.(Int64, x.user),
        convert.(Int64, x.item),
        convert.(Float32, x.rating),
    )
end;

In [ ]:
const training = RatingsDataset64(get_split("training", implicit))
const validation = get_split("validation", implicit);

## Alternating Least Squares Algorithm
* If $A$ is fixed then $U$ can be found by doing a regression on $A^{\Omega_i} u_i = R^{\Omega_i}$ with $L_2$ regularization $\lambda_u$ and weights $W$
* Alternating between fixing $A$ to solve $U$ and fixing $U$ to solve $A$ converges to a solution
* $\Omega_i = \{(i', j) \in \Omega | i' = i \}$ is projection of $\Omega$ onto the $i$th user

In [ ]:
function make_prediction(users, items, U, A)
    r = zeros(eltype(U), length(users))
    @views Threads.@threads for i = 1:length(r)
        if (users[i] <= size(U)[1]) && (items[i] <= size(A)[1])
            r[i] = dot(U[users[i], :], A[items[i], :])
        end
    end
    r
end;

In [ ]:
function calc_loss(df, U, A, weights)
    # truth = df.rating
    # pred = make_prediction(df.user, df.item, U, A)
    # β = (pred .* sqrt.(weights)) \ (truth .* sqrt.(weights))
    # loss = mse(truth, pred .* β, weights)
    # @info "loss: $loss β: $β"
    r = make_prediction(df.user, df.item, U, A)
    loss = residualized_loss(residual_alphas, implicit, r)
    @info "loss: $loss"
    loss
end;

In [ ]:
function ridge_regression(X, y, λ)
    (Matrix(X'X) + λ * LinearAlgebra.I(size(X)[2])) \ Vector(X'y)
end;

In [ ]:
# julia matrices are column major by default so we take adjoints to make them row major
@memoize function sparse_csr(i, j, v, m, n)
    sparse(j, i, v, n, m)'
end;

@memoize function gaussian_init_csr(source, K, el_type)
    Random.seed!(20220515 * hash(source) * K)
    (zeros(el_type, K, maximum(source)) + randn(K, maximum(source)) * K^(-1 / 4))'
end;

In [ ]:
function sparse_subset(A, rows)
    # returns a sparse matrix B such that: 
    # size(B) == size(A), B[rows, :] == A[rows, :], and B[~rows, :] == 0
    K = size(A)[2]
    nzval = vec(A[rows, :])
    rowval = repeat(rows, K)
    colptr = [1 + (x - 1) * length(rows) for x = 1:K+1]
    SparseMatrixCSC(size(A)..., colptr, rowval, nzval)
end;

In [ ]:
function update_users!(users, items, ratings, weights, U, A, λ_u, λ_w)
    R = sparse_csr(users, items, ratings, size(U)[1], size(A)[1])
    W = sparse_csr(users, items, weights, size(U)[1], size(A)[1])
    @tprogress Threads.@threads for i = 1:size(U)[1]
        w = expdecay.(W[i, :], λ_w / 2)
        X = sparse_subset(A, rowvals(R[i, :])) .* w
        y = R[i, :] .* w
        U[i, :] = ridge_regression(X, y, λ_u)
    end
end;

In [ ]:
function train_model(training, validation, λ_u, λ_a, λ_wu, λ_wa, K, stop_criteria)
    @info "training model with parameters [$λ_u, $λ_a, $λ_wu, $λ_wa]"
    users, items, ratings = training.user, training.item, training.rating
    U = copy(gaussian_init_csr(users, K, eltype(λ_u)))
    A = copy(gaussian_init_csr(items, K, eltype(λ_a)))
    wu = get_counts("training", implicit)
    wa = get_counts("training", implicit; by_item = true)
    function calc_losses()
        calc_loss(validation, U, A, get_weights("validation", implicit, "inverse"))
    end


    loss = Inf
    while !stop!(stop_criteria, loss)
        update_users!(users, items, ratings, wa, U, A, λ_u, log(λ_wa))
        update_users!(items, users, ratings, wu, A, U, λ_a, log(λ_wu))
        loss = calc_losses()
    end
    U, A, loss
end;

## Training

In [ ]:
function validation_mse(λ, K)
    λ = exp.(λ) # ensure λ is nonnegative
    # stop early so we can spend more computation exploring the parameter space
    stop_criteria = early_stopper(max_iters = 10)
    U, A, loss = train_model(training, validation, λ..., K, stop_criteria)
    loss
end;

In [ ]:
function optimize_model(K)
    # Find the best regularization hyperparameters
    res = Optim.optimize(
        λ -> validation_mse(λ, K),
        randn(4),
        Optim.LBFGS(),
        autodiff = :forward,
        Optim.Options(show_trace = true, extended_trace = true),
    )
    λ = exp.(Optim.minimizer(res))
    @info "The optimal λ is $λ, found in " * repr(Optim.f_calls(res)) * " function calls"

    # train model
    stop_criteria =
        early_stopper(max_iters = 100, patience = 5, min_rel_improvement = 0.0001)
    U, A, loss = train_model(training, validation, λ..., K, stop_criteria)

    # save model
    outdir = "$name.$K"
    model(users, items) = make_prediction(users, items, U, A)
    write_predictions(model, outdir = outdir, residual_alphas = residual_alphas)
    write_params(
        Dict("U" => U, "A" => A, "λ" => λ, "K" => K, "residual_alphas" => residual_alphas),
        outdir = outdir,
    )
end;

In [ ]:
for K in [2^3,2^4,2^5]
    optimize_model(K)
end;